In [1]:
import numpy as np
import pickle

with open("txt_data", "rb") as fp:   #Pickling
    data = pickle.load(fp)
with open("txt_labels", "rb") as fp:   #Pickling
    labels = pickle.load(fp)

print(len(data), len(labels))

31702 31702


In [2]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size = 0.30, shuffle=True, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, shuffle=True, random_state = 33)

print('split done.')

split done.


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, use_idf=True, stop_words='english')
vectorizer.fit(data)
x_train = vectorizer.transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()
x_valid = vectorizer.transform(x_valid).toarray()
print('> tfidVectorization done.')

> tfidVectorization done.


In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

params = {
    'kernel':['poly'],
    'degree':[1,2,3,4,5]
}
clf = GridSearchCV(SVC(), param_grid=params, cv=2, n_jobs=-1, verbose=1)
clf.fit(x_train, y_train)

clf = clf.best_estimator_
print()
print('Best params: ',clf)

y_pred = clf.predict(x_test)

print()
dt = pd.DataFrame(confusion_matrix(y_test, y_pred))
display(dt)
print(classification_report(y_test, y_pred, zero_division=0))

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 14.1min finished



Best params:  SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)



,0,1,2,3,4,5,6,7,8
0,867,22,25,28,50,5,11,42,10
1,18,1055,31,22,70,0,8,26,5
2,29,36,263,9,71,3,1,13,5
3,27,29,11,244,69,0,4,20,4
4,25,50,26,19,984,6,2,32,9
5,21,16,6,9,47,333,1,9,11
6,2,2,0,0,3,0,1089,138,167
7,4,5,0,0,15,0,101,1135,467
8,4,2,0,0,7,0,139,407,1085


              precision    recall  f1-score   support

           0       0.87      0.82      0.84      1060
           1       0.87      0.85      0.86      1235
           2       0.73      0.61      0.66       430
           3       0.74      0.60      0.66       408
           4       0.75      0.85      0.80      1153
           5       0.96      0.74      0.83       453
           6       0.80      0.78      0.79      1401
           7       0.62      0.66      0.64      1727
           8       0.62      0.66      0.64      1644

    accuracy                           0.74      9511
   macro avg       0.77      0.73      0.75      9511
weighted avg       0.75      0.74      0.74      9511



In [5]:
import pickle

with open('CLF_svm_poly','wb') as f:
    pickle.dump(clf, f)